<a href="https://colab.research.google.com/github/jeromepatel/DL-University-Course-Practicals/blob/master/18BCE088_DL_practical_4_Keras_CNN_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Feb 24 08:28:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

#importing libraries
from keras.datasets.cifar10 import load_data
from matplotlib import pyplot
from tensorflow import keras
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
batch_size = 512
num_classes = 10
epochs = 30


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
(X_train, Y_train), (X_test, Y_test) = load_data()
X_train.shape, X_test.shape

170500096/170498071 [==============================] - 2s 0us/step


((50000, 32, 32, 3), (10000, 32, 32, 3))

In [ ]:
#@title Reshaping, Unnecessary for this pract
#no need to reshape as we are working with cnn
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])

# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])
# X_train.shape, X_test.shape

In [ ]:
#@title Changing channel n dim (again not required)
# input image dimensions
# img_rows, img_cols = 32, 32
# if K.image_data_format() == 'channels_first':
#     X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
#     X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
#     input_shape = (3, img_rows, img_cols)
# else:
#     X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
#     X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
#     input_shape = (img_rows, img_cols, 3)



In [ ]:
X_train = X_train/255   #in range (0,1) now 
X_test = X_test/255
print(X_train[0].shape)
Y_train[0]

(32, 32, 3)


array([6], dtype=uint8)

In [ ]:
img_rows, img_cols = 32, 32
y_train,y_test = Y_train,Y_test
input_shape = (img_rows, img_cols, 3)
# convert class vectors to binary class matrices
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),strides = (1,1),padding = 'same',activation='relu',input_shape=input_shape))
model.add(Dropout(rate=0.3))
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(Dropout(rate=0.3))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(Dropout(rate=0.4))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=264, kernel_size=(3,3), strides=1, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dropout(rate=0.4))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=num_classes, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)       

In [ ]:
model.compile(Adam(lr = 0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
%%time
training_history = None
try:
   training_history =  model.fit(
    X_train,
        Y_train,
        validation_data = (X_test,Y_test),
        batch_size = batch_size,
        epochs = epochs,
        shuffle = True,
        verbose = 1
    )
except KeyboardInterrupt as e:
    print("\njust wait to complete training!!\n")

Epoch 1/15
98/98 [==============================] - 21s 210ms/step - loss: 2.6676 - accuracy: 0.2229 - val_loss: 8.3864 - val_accuracy: 0.1000
Epoch 2/15
98/98 [==============================] - 20s 204ms/step - loss: 1.6702 - accuracy: 0.3972 - val_loss: 5.6815 - val_accuracy: 0.1667
Epoch 3/15
98/98 [==============================] - 20s 204ms/step - loss: 1.4105 - accuracy: 0.4947 - val_loss: 3.5466 - val_accuracy: 0.1475
Epoch 4/15
98/98 [==============================] - 20s 204ms/step - loss: 1.2255 - accuracy: 0.5640 - val_loss: 3.8609 - val_accuracy: 0.1824
Epoch 5/15
98/98 [==============================] - 20s 204ms/step - loss: 1.0817 - accuracy: 0.6153 - val_loss: 3.9945 - val_accuracy: 0.1901
Epoch 6/15
98/98 [==============================] - 20s 204ms/step - loss: 0.9698 - accuracy: 0.6591 - val_loss: 2.6545 - val_accuracy: 0.3468
Epoch 7/15
98/98 [==============================] - 20s 204ms/step - loss: 0.8909 - accuracy: 0.6878 - val_loss: 2.3179 - val_accuracy: 0.4439

In [ ]:
#prediction using test set
score = model.evaluate(X_test, Y_test, verbose=0)
print(f'accuracy score for the Cifar10 classification of the dataset is : {score[1]}')

accuracy score for the Cifar10 classification of the dataset is : 0.7580999732017517
